<a href="https://colab.research.google.com/github/mohdahmad242/PID-detection/blob/main/EDA_new_batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import Library 

In [ ]:
import json
import pandas as pd
import cv2
import random
import os
from os import listdir
from os.path import isfile, join

# New annotations

In [ ]:
annotation_dir = "/content/drive/MyDrive/Official Folder DigitalTwins/Data/coco_converted/annots"
img_anno = [join(annotation_dir, f) for f in listdir(annotation_dir) if isfile(join(annotation_dir, f))]

In [ ]:
final_anno = []
for file in img_anno:
  with open(file) as f:
    anno = json.load(f)
  final_anno.extend(anno['annotations'])

In [ ]:
df = pd.DataFrame(final_anno)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6373 entries, 0 to 6372
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area          4316 non-null   float64
 1   bbox          6373 non-null   object 
 2   category_id   6373 non-null   int64  
 3   annot_id      6373 non-null   int64  
 4   image_id      6373 non-null   int64  
 5   iscrowd       6373 non-null   int64  
 6   segmentation  4316 non-null   object 
 7   classId       6373 non-null   object 
 8   className     6373 non-null   object 
 9   value         6373 non-null   object 
dtypes: float64(1), int64(4), object(5)
memory usage: 498.0+ KB


In [ ]:
df.head()

,area,bbox,category_id,annot_id,image_id,iscrowd,segmentation,classId,className,value
0,24226.470188,"[2065.2572, 4933.39, 156.34539999999998, 154.9...",0,0,0,0,"[2065.2572, 4933.39, 2221.6026, 4933.39, 2221....",303a720d,Instrument,
1,25591.603699,"[3375.9372, 4858.5614000000005, 161.0264, 158....",0,1,0,0,"[3375.9372, 4858.5614000000005, 3536.9636, 485...",ce708ed0,Instrument,
2,33076.899801,"[1715.1184, 4926.1058, 192.8572, 171.509799999...",0,2,0,0,"[1715.1184, 4926.1058, 1907.9756, 4926.1058, 1...",88588391,text,41LAL 14
3,15912.918696,"[4805.5145999999995, 4279.1364, 129.1955999999...",0,3,0,0,"[4805.5145999999995, 4279.1364, 4934.710199999...",af6b0c8f,Unidentified,
4,7769.853904,"[3721.395, 3626.2072, 141.3662, 54.9626]",0,4,0,0,"[3721.395, 3626.2072, 3862.7612, 3626.2072, 38...",d6952e31,Valve,


In [ ]:
df.className.unique().shape

(18,)

In [ ]:
line = df[df.className == 'line']

In [ ]:
line.head()

,area,bbox,category_id,annot_id,image_id,iscrowd,segmentation,classId,className,value
98,NaN,"[4722.1928, 4301.651199999999, 87.0666, 62.2468]",2,98,0,0,None,2eb6393d,line,
99,NaN,"[4722.1928, 4301.651199999999, 87.0666, 62.2468]",2,99,0,0,None,5416f233,line,
100,NaN,"[4722.1928, 4301.651199999999, 87.0666, 62.2468]",2,100,0,0,None,6372b6b0,line,
101,NaN,"[4722.1928, 4301.651199999999, 87.0666, 62.2468]",2,101,0,0,None,0862930c,line,
102,NaN,"[4722.1928, 4301.651199999999, 87.0666, 62.2468]",2,102,0,0,None,32ce779b,line,


In [ ]:
df.className.unique()

array(['Instrument', 'text', 'Unidentified', 'Valve', 'Flow_Direction',
       'Plug', 'Flange', 'Tank_Vessel', 'unidentified', 'line', 'Pump',
       'Engine', 'Line_Break', 'Actuator', 'Connector', 'Concentric',
       'Separator', 'Flange_Blind'], dtype=object)

In [ ]:
df.className = df.className.str.lower()

In [ ]:
df.className.unique(), df.className.unique().shape

(array(['instrument', 'text', 'unidentified', 'valve', 'flow_direction',
        'plug', 'flange', 'tank_vessel', 'line', 'pump', 'engine',
        'line_break', 'actuator', 'connector', 'concentric', 'separator',
        'flange_blind'], dtype=object), (17,))

# Old Annotations

In [ ]:
with open("/content/PID (1).json") as f:
  ann = json.load(f)

In [ ]:
ann.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [ ]:
old_df = pd.DataFrame(ann['annotations'])

In [ ]:
old_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6225 entries, 0 to 6224
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            6225 non-null   int64 
 1   image_id      6225 non-null   int64 
 2   segmentation  6225 non-null   object
 3   iscrowd       6225 non-null   int64 
 4   bbox          6225 non-null   object
 5   area          6225 non-null   int64 
 6   category_id   6225 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 340.6+ KB


In [ ]:
6373+6225

12598

In [ ]:
def add_class(col):
  for i in ann['categories']:
    if i['id'] == col:
      return i['name'].lower()

In [ ]:
old_df['className'] = old_df['category_id'].apply(add_class)

In [ ]:
old_df.head()

,id,image_id,segmentation,iscrowd,bbox,area,category_id,className
0,1,1,"[[5130.92, 3420.16, 5130.92, 3529.81, 5241.13,...",0,"[5130.92, 3420.16, 110.21000000000004, 109.650...",385600,352833,instrument
1,2,1,"[[3520.11, 3419.77, 3520.11, 3529.25, 3630.55,...",0,"[3520.11, 3419.77, 110.44000000000005, 109.480...",386350,352833,instrument
2,3,1,"[[2655.96, 3419.43, 2655.96, 3529.17, 2764.54,...",0,"[2655.96, 3419.43, 108.57999999999993, 109.740...",379777,352833,instrument
3,4,1,"[[3640.81, 3074.69, 3640.81, 3118.19, 3682.77,...",0,"[3640.81, 3074.69, 41.960000000000036, 43.5]",127764,352842,concentric
4,5,1,"[[3466.15, 3282.72, 3466.15, 3341.81, 3522.46,...",0,"[3466.15, 3282.72, 56.309999999999945, 59.0900...",184894,352845,line_break


In [ ]:
print((old_df.className.unique())), print(df.className.unique())

['instrument' 'concentric' 'line_break' 'flow_direction' 'unidentified'
 'inside text' 'text' 'tank_vessel' 'valve' 'flange' 'actuator'
 'connector' 'separator' 'engine' 'plug' 'flange_blind' 'piping_lines'
 'other_lines']
['instrument' 'text' 'unidentified' 'valve' 'flow_direction' 'plug'
 'flange' 'tank_vessel' 'line' 'pump' 'engine' 'line_break' 'actuator'
 'connector' 'concentric' 'separator' 'flange_blind']


(None, None)

In [ ]:
old = old_df.className.unique().tolist()
old.sort()
new = df.className.unique().tolist()
new.sort()

In [ ]:
old, len(old)

(['actuator',
  'concentric',
  'connector',
  'engine',
  'flange',
  'flange_blind',
  'flow_direction',
  'inside text',
  'instrument',
  'line_break',
  'other_lines',
  'piping_lines',
  'plug',
  'separator',
  'tank_vessel',
  'text',
  'unidentified',
  'valve'],
 18)

In [ ]:
Total number of classes in old annotations are 18,
These are the classes that old have but not there in new one -> "inside text", "other_lines", and "piping_lines".
Total number of classes in New annotations are 17,
Classes that are present in new annotations but are not in old one -> "line" and "pump"

My suggestion is to combine "other_lines" and "piping_lines" to create one "line" class similar to New annotation. 
And also combine "inside text" class with "text" class in old annotation. 
After that we can use both annotations combined. 

In [ ]:
new, len(new)

(['actuator',
  'concentric',
  'connector',
  'engine',
  'flange',
  'flange_blind',
  'flow_direction',
  'instrument',
  'line',
  'line_break',
  'plug',
  'pump',
  'separator',
  'tank_vessel',
  'text',
  'unidentified',
  'valve'],
 17)

In [ ]:
class_comp = {
    "old": old,
    "new": new
}